In [16]:

import pandas as pd
import numpy as np
import re


In [17]:
wikidata = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", 
             match='Postcode', header=0)[0]

In [18]:
wikidata = wikidata.rename(
    mapper={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'},
    axis=1)
wikidata.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [19]:
befrows = wikidata.shape[0]
wikidata = wikidata[wikidata['Borough']!='Not assigned']
assert wikidata.shape[0] < befrows, "No rows were removed"

In [20]:
grouped = wikidata.groupby(['PostalCode','Borough'], 
                           as_index=False).aggregate(lambda x: ', '.join(x))

In [21]:
for index, row in grouped.iterrows():
    row['Neighborhood'] = row['Borough'] if re.search('Not assigned', row['Neighborhood']) else row['Neighborhood']

In [22]:
grouped.shape

(103, 3)

section 2

In [23]:
import geocoder

ModuleNotFoundError: No module named 'geocoder'

In [24]:
geodata = pd.read_csv("https://cocl.us/Geospatial_data").rename(
    mapper={"Postal Code": "PostalCode"}, axis=1)

In [25]:
geodata['PostalCode'] = geodata['PostalCode'].apply(str)
grouped['PostalCode'] = grouped['PostalCode'].apply(str)
merged = grouped.merge(geodata, on=['PostalCode'])
merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


section 3

In [26]:
tordata = merged[merged['Borough'].str.contains('Toronto')]

In [27]:
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

ModuleNotFoundError: No module named 'folium'

In [28]:
address = 'Toronto, ONT'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#adjust
longitude +=.25
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(tordata['Latitude'], 
                                           tordata['Longitude'], 
                                           tordata['Borough'], 
                                           tordata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

NameError: name 'Nominatim' is not defined

In [30]:
with open('/Users/roshani/Desktop/IBM/foursquarecreds.txt') as f:
    fsq = f.read()
client_id = re.search("(?<=CLIENT_ID)[ =]+'([A-Z0-9]+)", fsq).group(1)
client_secret = re.search("(?<=CLIENT_SECRET)[ =]+'([A-Z0-9]+)", fsq).group(1)
#not sure if coursera wants me to update this or not
VERSION = '20180605' # Foursquare API version

FileNotFoundError: [Errno 2] No such file or directory: '/Users/roshani/Desktop/IBM/foursquarecreds.txt'